In [3]:
import pandas as pd
import numpy as np
import csv, json
from urllib.parse import urlparse

In [4]:
def getID(url):
    parsed_url = urlparse(url)
    path_segments = parsed_url.path.split('/')
    return int(path_segments[3])
#getID("https://www.tiktokv.com/share/video/7315561816673750318/")

In [5]:
def processCsv(csvd):
    csvdf = csvd
    position = csvdf.columns.get_loc('video_timestamp') + 1
    csvdf.insert(position, 'video_time_watched', np.nan)
    return csvdf

In [6]:
def processJson(jsond):
    jsondf = jsond
    jsondf['video_id'] = jsondf['Link'].apply(getID)
    return jsondf

In [7]:
#for ech pair create new df with date
def merged_date(csvd, jsond):
    csvdf = csvd
    jsondf = jsond
    csvrow = 0
    jsonrow = 0
    for id in jsondf['video_id']: #for each video watched in json
        if id == csvdf['video_id'][csvrow]: #if the id for the json equals the next csv row
            csvdf.loc[csvrow, 'video_time_watched'] = jsondf.loc[jsonrow, 'Date']
            if csvrow < len(csvdf)-1:
                csvrow = csvrow+1   #go on to the next csv row
        jsonrow = jsonrow + 1  #go to the next json no matter if match or not
    return csvdf



In [10]:
#processjson and csv and insert
def mergefile(csv,json):
    csvfile = pd.read_csv(f'{csv}')
    jsonfile = pd.read_json(f'{json}')
    return merged_date(processCsv(csvfile), processJson(jsonfile))
    
#sec1gr2_csv = pd.read_csv('pyktok_data/Sec1Gr2_26301.csv') #come back label wrong
#sec1gr2_json = pd.read_json('user_jsons/Sec2Gr2_26301.json')

In [11]:
sec1gr2_final = mergefile('pyktok_data/Sec1Gr2_26301.csv','user_jsons/Sec2Gr2_26301.json')
sec1gr2_final

/var/folders/ls/b1_wv00s5r5fjstz103_p3rh0000gn/T/ipykernel_31834/966991795.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-03-06 19:44:41' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  csvdf.loc[csvrow, 'video_time_watched'] = jsondf.loc[jsonrow, 'Date']


,video_id,video_timestamp,video_time_watched,video_duration,video_locationcreated,suggested_words,video_diggcount,video_sharecount,video_commentcount,video_playcount,...,video_is_ad,video_stickers,author_username,author_name,author_followercount,author_followingcount,author_heartcount,author_videocount,author_diggcount,author_verified
0,7315561816673750318,2023-12-22T18:16:20,2024-03-06 19:44:41,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,...,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False
1,7315561816673750318,2023-12-22T18:16:20,2024-03-06 18:00:25,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,...,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False
2,7315561816673750318,2023-12-22T18:16:20,2024-03-06 17:58:36,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,...,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False
3,7315561816673750318,2023-12-22T18:16:20,2024-03-06 17:56:52,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,...,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False
4,7315561816673750318,2023-12-22T18:16:20,2024-03-06 17:46:03,18.0,US,"flipnote, flipnote animation, asdf movie, flip...",3500000.0,39500.0,10800.0,18700000.0,...,False,NaN,zanelittlemusic,Zane Little,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8063,7321132514330905899,2024-01-06T18:33:34,2024-02-20 01:57:54,58.0,US,"fake awake by tarte, Tarte Cosmetics, tubing m...",107500.0,1162.0,521.0,18500000.0,...,True,NaN,michellesays,Michelle,NaN,NaN,NaN,NaN,NaN,False
8064,7331962685812215042,2024-02-04T22:59:56,2024-02-20 01:57:51,6.0,FAKE-AD,"Body Deodorant, Deodorant For Body Odor, Deodo...",5589.0,118.0,92.0,10600000.0,...,True,NaN,Secret,Secret,NaN,NaN,NaN,NaN,NaN,False
8065,7335549375667653934,2024-02-14T14:58:41,2024-02-20 01:57:51,64.0,US,"Bk Beauty Brushes, bk beauty, concealer brush,...",23200.0,113.0,205.0,1300000.0,...,True,NaN,itsmakeupbymelissam,Melissa Murdick,NaN,NaN,NaN,NaN,NaN,True
8066,7337225129354661153,2024-02-19T03:21:02,2024-02-20 01:57:38,62.0,GB,NaN,12600.0,84.0,439.0,3200000.0,...,False,NaN,ethancaf,EthanC24,NaN,NaN,NaN,NaN,NaN,False


In [174]:
new = processJson(sec1gr2_json)
id = 7328816594363338030
indices = new.query('video_id == 7328816594363338030').index.tolist()
print(indices)
#print(csv.duplicated(subset='video_id').sum())

[13855]
